In [1]:
import pandas as pd

In [2]:
demo_data = pd.read_stata("demographic_data.dta")["fips_tract_10"]

In [3]:
demo_data.head()

0    1.001020e+09
1    1.001020e+09
2    1.001020e+09
3    1.001020e+09
4    1.001020e+09
Name: fips_tract_10, dtype: float64

In [4]:
test_data = pd.read_csv("Alachua_County_Dispatch_2009_geo_RR.csv")

In [5]:
from constants import list_off, time_name, dispatch_name, prio_name, desc_name,call_name

In [6]:
# # standardize variable names for San Diego
# if strpos("Greensboro", "`city'") > 0 {
#     df["priority_1"] = df[desc_name_`cityname'}
# }

# # standardize description variable name for St. Paul 
# if "St. Paul" in city > 0:
#     cap rename description ${desc_name_`cityname'}

In [44]:
def determine_officer_initiated(df, city):
    officer_initiated = 0
    useless = ["assist","warrant","chase","foot","pursuit","traffic","ETS","routine","executive","parking","walk-in","fire","mark-out","driving","truant"]
    useless.extend(list_off[city])
    
    description_col = desc_name[city]
    
    if "N/A" not in description_col:
        description = df[description_col]
        for word in useless:
            if (all(item not in description.lower() for item in ["fired", "gunfire", "firework"])
                and word in description.lower()):
                    officer_initiated = 1
            
    if "N/A" not in dispatch_name[city] and any(item in city for item in ["Detroit","Los Angeles","Mesa","Portland","Sandy","Springs"]):
        if time_name[city] == dispatch_name[city]:
            officer_initiated = 1 

    if "Elgin" in city:
        if "F" in prio_name[city]:
            officer_initiated = 1 

    if "Harlingen" in city:
        officer_initiated = 0
        if callsource.lower() == "officer initiated":
            officer_initiated = 1 
    
    if "Richmond" in city:
        if df["isselfinitiated"] == "No": 
            officer_initiated = 0 
        if df["isselfinititated"] == "No": 
            officer_initiated = 0
        if df["isselfinitiated"] == "Yes":
            officer_initiated = 1
        if df["isselfinititated"] == "Yes": 
            officer_initiated = 1 

    if "Norwalk" in city:
        officer_initiated = 0
        if "officer-initiated" in dispo_name["cityname"]:
            officer_initiated = 1

    if "Reno" in city or "Sparks" in city:
        if description.lower() == "t":
            officer_initiated = 1
    
    if "San Mateo" in city: 
        officer_initiated = 1 if df["sourceofcall"] == "Officer Initiated" else 0
    
    if "Tacoma" in city:
        officer_initiated = 1 if df["on_view"] == 1 else 0
        
    return officer_initiated

In [45]:
test_data.columns = [x.lower().replace(" ", "_") for x in test_data.columns]
test_data["officer_initiated"] = test_data.apply(determine_officer_initiated, city="Alachua_County", axis=1)
len(test_data.loc[test_data["officer_initiated"] == 0])

61417

In [48]:
len(test_data.loc[test_data["officer_initiated"] == 0].merge(demo_data, left_on="fips", right_on="fips_tract_10"))/61417

0.9449663773873683

In [49]:
pd.read_stata("alachua_county_dispatch_2009_geo_merge_rates.dta")

,city,year,filename,merge_rate,total_overall,state
0,Alachua County,2009.0,alachua_county_dispatch_2009_geo_domvio_officer,0.944966,61417.0,FL
